In [1]:
import duckdb

In [2]:
def query(q):
    db = duckdb.connect("../data/chess.duckdb")
    db.sql("SET TimeZone = 'UTC'")
    df = db.sql(q).to_df()
    db.close()
    return df

In [3]:
query('select * from information_schema.tables')

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,chess,chess_data_prep,game_moves,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,chess,chess_data_prep,player_games,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,chess,chess_data_raw,players_games,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,chess,chess_data_raw,_dlt_loads,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,chess,chess_data_raw,_dlt_pipeline_state,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,chess,chess_data_raw,_dlt_version,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,chess,chess_data_staging,players_games,VIEW,None,None,None,None,None,NO,NO,None,None


In [27]:
print(query('select pgn from chess_data_prep.player_games limit 5')['pgn'][0])

[Event "Live Chess"]
[Site "Chess.com"]
[Date "2024.05.01"]
[Round "-"]
[White "Christophe781994"]
[Black "johnnywhoopp"]
[Result "1/2-1/2"]
[CurrentPosition "4k3/8/5Q2/P1P1B2P/1P2P3/7B/2K5/6R1 b - -"]
[Timezone "UTC"]
[ECO "A00"]
[ECOUrl "https://www.chess.com/openings/Van-Geet-Opening-1...c5"]
[UTCDate "2024.05.01"]
[UTCTime "09:45:03"]
[WhiteElo "600"]
[BlackElo "591"]
[TimeControl "600"]
[Termination "Game drawn by stalemate"]
[StartTime "09:45:03"]
[EndDate "2024.05.01"]
[EndTime "09:54:57"]
[Link "https://www.chess.com/game/live/107152503720"]

1. Nc3 {[%clk 0:09:59.9]} 1... c5 {[%clk 0:09:55.9]} 2. h3 {[%clk 0:09:58.2]} 2... d6 {[%clk 0:09:51.7]} 3. g4 {[%clk 0:09:56.9]} 3... g6 {[%clk 0:09:32]} 4. f4 {[%clk 0:09:54.5]} 4... Bg7 {[%clk 0:09:29.7]} 5. g5 {[%clk 0:09:50.6]} 5... Nc6 {[%clk 0:09:19.5]} 6. h4 {[%clk 0:09:49.6]} 6... Qd7 {[%clk 0:09:04.4]} 7. e4 {[%clk 0:09:48.4]} 7... b6 {[%clk 0:08:49.4]} 8. Nf3 {[%clk 0:09:44.7]} 8... Bb7 {[%clk 0:08:47.3]} 9. a3 {[%clk 0:09:42.9]

In [29]:
query("""
with game_moves_pivot as (
    PIVOT chess_data_prep.game_moves
    ON color_move
    USING sum(move_time_seconds) as total_move_time, count(1) as num_moves
    group by uuid
)
, player_games as (
    select
    *
    from chess_data_prep.player_games
)
, joined as (
    select
    pg.*
    , white_total_move_time
    , white_num_moves
    , black_total_move_time
    , black_num_moves
    from player_games as pg
    
    left join game_moves_pivot as gm
    on pg.uuid = gm.uuid
)
, final as (
    select
    *

    -- PLAYER DETAILS
    , if(player_color = 'White', white_total_move_time, black_total_move_time) as player_total_move_time
    , if(player_color = 'White', white_num_moves, black_num_moves) as player_num_moves

    -- OPPONENT DETAILS
    , if(player_color = 'White', black_total_move_time, white_total_move_time) as opponent_total_move_time
    , if(player_color = 'White', black_num_moves, white_num_moves) as opponent_num_moves

    -- GAME TIME DETAILS
    , cast(replace(regexp_extract(pgn, '(UTCDate )"(.*)"', 2), '.', '-') as date) as game_start_date
    , regexp_extract(pgn, '(StartTime )"(.*)"', 2) as game_start_time
    , cast(concat(game_start_date, ' ', game_start_time) as timestamp) as game_start_timestamp
    , cast(end_time as timestamp) as game_end_timestamp
    , age(game_end_timestamp, game_start_timestamp) as time_played_interval
    , epoch(time_played_interval) as time_played_seconds

    from joined
)
select
-- ID
uuid

-- GAME
, url
, rated
, rules
, time_class

-- TIME
, time_control
, time_control_base
, time_control_add_seconds
, time_played_seconds
, game_start_timestamp
, game_end_timestamp

-- WHITE-BLACK
, white__uuid
, white__username
, white__aid
, white__rating
, white__result
, white_total_move_time
, white_num_moves

, black__uuid
, black__username
, black__aid
, black__rating
, black__result
, black_total_move_time
, black_num_moves

-- PLAYER-OPPONENT
, player_color
, player_rating
, player_result
, player_total_move_time
, player_num_moves
, opponent_rating
, opponent_result
, opponent_total_move_time
, opponent_num_moves
, is_stronger_opponent
, player_wdl
, player_wdl_reason

-- BOARD
, initial_setup
, fen
, pgn
, pgn_moves
, pgn_move_extract
, pgn_clock_extract
, eco
, eco_url
, eco_name
, checkmate_pieces

-- MISC
, tcn
, accuracies__white
, accuracies__black

from final
""")

,uuid,url,time_class,time_control,rated,rules,time_control_base,time_control_add_seconds,time_played_seconds,game_start_timestamp,...,pgn_moves,pgn_move_extract,pgn_clock_extract,eco,eco_url,eco_name,checkmate_pieces,tcn,accuracies__white,accuracies__black
0,2c82f063-579f-11ef-ac6e-00a8d701000f,https://www.chess.com/game/2c82f063-579f-11ef-...,rapid,600,True,chess,600,0,545.0,2024-08-11 05:04:23,...,1. e4 {[%clk 0:09:59.9]} 1... e6 {[%clk 0:09:5...,"[1. e4, 1... e6, 2. Nf3, 2... d6, 3. Bc4, 3......","[{[%clk 0:09:59.9]}, {[%clk 0:09:58.9]}, {[%cl...",C00,https://www.chess.com/openings/French-Defense-...,French-Defense-Knight-Variation,"[queen, rook]",mC0SgvZRfA5Qeg3Nfe2MksMEeuEvuvNFlB92vDSKDvKBA1...,NaN,NaN
1,56cb4924-f198-11ee-ae34-61d20601000f,https://www.chess.com/game/56cb4924-f198-11ee-...,rapid,600,True,chess,600,0,782.0,2024-04-03 08:58:28,...,1. e4 {[%clk 0:09:56.6]} 1... f6 {[%clk 0:09:5...,"[1. e4, 1... f6, 2. d4, 2... e5, 3. Nc3, 3... ...","[{[%clk 0:09:56.6]}, {[%clk 0:09:55.3]}, {[%cl...",B00,https://www.chess.com/openings/Kings-Pawn-Open...,Kings-Pawn-Opening-Barnes-Defense-2.d4,[],mC1TlB0KbsZRfHYQHARJAr9zclzGdv!0ec8!gm0UCJQJrJ...,NaN,NaN
2,794b69e0-41eb-11ef-9e1d-500c8901000f,https://www.chess.com/game/794b69e0-41eb-11ef-...,rapid,600,True,chess,600,0,1054.0,2024-07-14 14:15:07,...,1. e4 {[%clk 0:09:59.9]} 1... c5 {[%clk 0:09:5...,"[1. e4, 1... c5, 2. Bc4, 2... e6, 3. Qf3, 3......","[{[%clk 0:09:59.9]}, {[%clk 0:09:58.2]}, {[%cl...",B20,https://www.chess.com/openings/Sicilian-Defens...,Sicilian-Defense-Bowdler-Attack-2...e6,[],mCYIfA0Sdv!TgmZRlt2UcM92bs8!MT2TsH5QtBIBksBsjs...,NaN,NaN
3,971183b8-639c-11ef-9e8d-b4769d01000f,https://www.chess.com/game/971183b8-639c-11ef-...,rapid,600,True,chess,600,0,821.0,2024-08-26 11:16:07,...,1. e4 {[%clk 0:09:59.4]} 1... c5 {[%clk 0:09:5...,"[1. e4, 1... c5, 2. d4, 2... cxd4, 3. Qxd4, 3....","[{[%clk 0:09:59.4]}, {[%clk 0:09:58.9]}, {[%cl...",B53,https://www.chess.com/openings/Sicilian-Defens...,Sicilian-Defense-Chekhover-Variation-4...Nc6-5...,[],mCYIlBIBdB5QBuZRgv!Tbs2UsJ92JT2TfA8!egT2fd0Sjr...,NaN,NaN
4,b0ea4633-692e-11ef-b07e-c236d801000f,https://www.chess.com/game/b0ea4633-692e-11ef-...,rapid,600,True,chess,600,0,415.0,2024-09-02 13:24:33,...,1. e4 {[%clk 0:09:57.3]} 1... c5 {[%clk 0:09:5...,"[1. e4, 1... c5, 2. Nf3, 2... Nc6, 3. Bb5, 3.....","[{[%clk 0:09:57.3]}, {[%clk 0:09:59.4]}, {[%cl...",B51,https://www.chess.com/openings/Sicilian-Defens...,Sicilian-Defense-Canal-Attack...4.d4-cxd4-5.Nx...,[],mCYIgv5QfHZRlBIBvB6Zeg!THQZQBQXQCKRKd747cM2UMT...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,f852cbac-0305-11ef-a7cf-1439b001000f,https://www.chess.com/game/f852cbac-0305-11ef-...,rapid,600,True,chess,600,0,847.0,2024-04-25 13:16:04,...,1. e4 {[%clk 0:09:59.5]} 1... c5 {[%clk 0:09:5...,"[1. e4, 1... c5, 2. Qf3, 2... d6, 3. Bc4, 3......","[{[%clk 0:09:59.5]}, {[%clk 0:09:58.9]}, {[%cl...",B20,https://www.chess.com/openings/Sicilian-Defense,Sicilian-Defense,"[knight, knight, rook, rook]",mCYIdvZRfA0Sgx!TxM70AH5QCKRKeg2Ufe92eKWOHA8!Ku...,NaN,NaN
520,efc58dd6-f190-11ee-ae34-61d20601000f,https://www.chess.com/game/efc58dd6-f190-11ee-...,rapid,600,True,chess,600,0,1157.0,2024-04-03 08:05:29,...,1. d4 {[%clk 0:09:50.6]} 1... d5 {[%clk 0:09:4...,"[1. d4, 1... d5, 2. e3, 2... Nf6, 3. Nc3, 3......","[{[%clk 0:09:50.6]}, {[%clk 0:09:40.3]}, {[%cl...",D00,https://www.chess.com/openings/Queens-Pawn-Ope...,Queens-Pawn-Opening-1...d5-2.e3-Nf6,[],lBZJmu!Tbs0SnD9ziqzRgv8!pF98hg5QoESKBKRKDKQKEM...,NaN,NaN
521,8c36989d-f32d-11ee-a90b-f3215801000f,https://www.chess.com/game/8c36989d-f32d-11ee-...,rapid,600,True,chess,600,0,759.0,2024-04-05 09:19:04,...,1. d4 {[%clk 0:09:57]} 1... Nf6 {[%clk 0:09:55...,"[1. d4, 1... Nf6, 2. Bg5, 2... g6, 3. Bxf6, 3....","[{[%clk 0:09:57]}, {[%clk 0:09:55.5]}, {[%clk ...",A45,https://www.chess.com/openings/Trompowsky-Atta...,Trompowsky-Attack-2...g6-3.Bxf6-exf6,[],lB!TcM2UMT0Tks92mC8!CKTKBKZRnD5QfAQGjrXHAH6SsA

In [ ]:
Index(['end_time', 'url', 'pgn', 'time_control', 'rated', 'tcn', 'uuid',
       'initial_setup', 'fen', 'time_class', 'rules', 'white__rating',
       'white__result', 'white__aid', 'white__username', 'white__uuid',
       'black__rating', 'black__result', 'black__aid', 'black__username',
       'black__uuid', 'accuracies__white', 'accuracies__black',
       'time_control_base', 'time_control_add_seconds', 'pgn_moves',
       'pgn_move_extract', 'pgn_clock_extract', 'eco', 'eco_url', 'eco_name',
       'player_color', 'player_rating', 'player_result', 'opponent_rating',
       'opponent_result', 'is_stronger_opponent', 'player_wdl',
       'player_wdl_reason', 'checkmate_pieces', 'White_total_move_time',
       'White_num_moves', 'Black_total_move_time', 'Black_num_moves',
       'player_total_move_time', 'player_num_moves',
       'opponent_total_move_time', 'opponent_num_moves', 'game_start_date',
       'game_start_time', 'game_start_timestamp', 'game_end_timestamp',
       'time_played_interval', 'time_played_seconds'],
      dtype='object')


In [27]:
query("""
with player_games as (
    select
    uuid
    , time_control_base
    , time_control_add_seconds
    , pgn_move_extract
    , pgn_clock_extract
    from chess_data_prep.player_games
    limit 10
)
, unnest as (
    select 
    *
    , generate_subscripts(pgn_move_extract, 1) AS game_move_index
    , unnest(pgn_move_extract) as move_unnest
    , unnest(pgn_clock_extract) as clock_unnest
    , split(move_unnest, ' ')[1] as color_move_index_raw
    , cast(regexp_replace(color_move_index_raw, '\.+', '') as int) as color_move_index
    , if(regexp_matches(color_move_index_raw, '\.\.\.'), 'Black', 'White') as color_move
    , split(move_unnest, ' ')[2] as move
    
    -- This is the clock after the addition of time
    , epoch(cast(replace(split(clock_unnest, ' ')[2], ']}}', '') as interval)) as clock_interval_post_move
    
    -- To get the clock before the addition of time
    , clock_interval_post_move - time_control_add_seconds as clock_interval_move
    
    from player_games
)
, final as (
    select 
    uuid
    , game_move_index
    , color_move
    , color_move_index
    , move
    , coalesce(
        lag(clock_interval_post_move) over(partition by uuid, color_move order by color_move_index)
        , time_control_base
    )  as prev_clock_interval
    , clock_interval_move
    , clock_interval_post_move
    , prev_clock_interval - clock_interval_move as move_time_seconds
    from unnest
)
select
concat(uuid::string, '_', game_move_index::string) as id
, uuid::string as uuid
, game_move_index::int as game_move_index
, color_move::string as color_move
, color_move_index::int as color_move_index
, move::string as move
, prev_clock_interval::double as prev_clock_interval
, clock_interval_move::double as clock_interval_move
, clock_interval_post_move::double as clock_interval_post_move
, move_time_seconds::double as move_time_seconds
from final
""")

,id,uuid,game_move_index,color_move,color_move_index,move,prev_clock_interval,clock_interval_move,clock_interval_post_move,move_time_seconds
0,aace8558-f19b-11ee-9440-128b6201000f_1,aace8558-f19b-11ee-9440-128b6201000f,1,White,1,e4,600.0,597.7,597.7,2.3
1,aace8558-f19b-11ee-9440-128b6201000f_3,aace8558-f19b-11ee-9440-128b6201000f,3,White,2,d4,597.7,591.7,591.7,6.0
2,aace8558-f19b-11ee-9440-128b6201000f_5,aace8558-f19b-11ee-9440-128b6201000f,5,White,3,Nc3,591.7,583.7,583.7,8.0
3,aace8558-f19b-11ee-9440-128b6201000f_7,aace8558-f19b-11ee-9440-128b6201000f,7,White,4,f3,583.7,576.2,576.2,7.5
4,aace8558-f19b-11ee-9440-128b6201000f_9,aace8558-f19b-11ee-9440-128b6201000f,9,White,5,Bg5,576.2,554.5,554.5,21.7
...,...,...,...,...,...,...,...,...,...,...
532,6531c514-f18c-11ee-b278-fbb15501000f_41,6531c514-f18c-11ee-b278-fbb15501000f,41,White,21,Nf6,403.8,395.8,395.8,8.0
533,6531c514-f18c-11ee-b278-fbb15501000f_43,6531c514-f18c-11ee-b278-fbb15501000f,43,White,22,Qg8+,395.8,367.5,367.5,28.3
534,6531c514-f18c-11ee-b278-fbb15501000f_45,6531c514-f18c-11ee-b278-fbb15501000f,45,White,23,Qh8+,367.5,336.6,336.6,30.9
535,6531c514-f18c-11ee-b278-fbb15501000f_47,6531c514-f18c-11ee-b278-fbb15501000f,47,White,24,Rhe1,336.6,315.3,315.3,21.3


In [7]:
query("select * from chess_data_raw.chess_openings where opening = 'unknown'")['url'][0]

'https://www.chess.com/openings/Nimzowitsch-Defense-Declined-2...Nf6'

In [20]:
query("""
    with dataset as (
        select 
        uuid
        , array_agg(move order by game_move_index) as lst
        from chess_data_staging.game_moves
        where uuid = '0019bf9d-62b3-11ef-9f40-a9fcd401000f'
        group by 1
    )
    select *, lst[1:2]
    from dataset
""")

,uuid,lst,lst[1:2]
0,0019bf9d-62b3-11ef-9f40-a9fcd401000f,"[e4, e5, Nf3, Qf6, Bc4, Bc5, O-O, d6, c3, Qg6,...","[e4, e5]"


In [19]:
query("""
    select 
    uuid,
    move
    from chess_data_staging.game_moves
    where uuid = '0019bf9d-62b3-11ef-9f40-a9fcd401000f'
    order by game_move_index
    limit 10
""")

,uuid,move
0,0019bf9d-62b3-11ef-9f40-a9fcd401000f,e4
1,0019bf9d-62b3-11ef-9f40-a9fcd401000f,e5
2,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Nf3
3,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Qf6
4,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Bc4
5,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Bc5
6,0019bf9d-62b3-11ef-9f40-a9fcd401000f,O-O
7,0019bf9d-62b3-11ef-9f40-a9fcd401000f,d6
8,0019bf9d-62b3-11ef-9f40-a9fcd401000f,c3
9,0019bf9d-62b3-11ef-9f40-a9fcd401000f,Qg6


In [7]:
username = 'johnnywhoopp'

q =f"""
with player_games as (
    select
    end_time
    , url 
    , pgn
    , time_control
    , rated
    , tcn
    , uuid
    , initial_setup
    , fen
    , time_class
    , rules
    , white__rating
    , white__result
    , white__aid
    , white__username
    , white__uuid
    , black__rating
    , black__result
    , black__aid
    , black__username
    , black__uuid
    , accuracies__white
    , accuracies__black
    from chess_data_staging.players_games
)
, final as (
    select
    *
    
    -- Time control details
    , cast(split(time_control, '+')[1] as integer) as time_control_base
    , cast(coalesce(split(time_control, '+')[2], '0') as integer) as time_control_add_seconds
    
    -- PGN details
    , split(pgn, '\n\n')[2] as pgn_moves
    , regexp_extract_all(pgn_moves, '\d+\.+ [\S]+') as pgn_move_extract
    , regexp_extract_all(pgn_moves, '{{\[%clk \S+\]}}') as pgn_clock_extract
    
    -- PGN ECO details
    , regexp_extract(pgn, '(ECO )"(.*)"', 2) as eco
    , regexp_extract(pgn, '(ECOUrl )"(.*)"', 2) as eco_url
    , replace(eco_url, 'https://www.chess.com/openings/', '') as eco_name
    
    -- PLAYER details
    , if(lower(white__username)='{username}', 'White', 'Black') as player_color
    , if(player_color = 'White', white__rating, black__rating) as player_rating
    , if(player_color = 'White', white__result, black__result) as player_result
    
    -- OPPONENT DETAILS
    , if(player_color = 'White', black__rating, white__rating) as opponent_rating
    , if(player_color = 'White', black__result, white__result) as opponent_result
    
    -- PLAYER-OPPONENT DETAILS
    , opponent_rating > player_rating as is_stronger_opponent
    , case 
        when player_result = 'win' then 'win'
        when opponent_result = 'win' then 'lose'
        when player_result <> 'win' and opponent_result <> 'win' then 'draw'
        else 'unknown'
    end as player_wdl
    , if(player_result='win', opponent_result, player_result) as player_wdl_reason
    
    from player_games
    limit 10
)
select * from final
"""

query(q)

,end_time,url,pgn,time_control,rated,tcn,uuid,initial_setup,fen,time_class,...,eco_url,eco_name,player_color,player_rating,player_result,opponent_rating,opponent_result,is_stronger_opponent,player_wdl,player_wdl_reason
0,2024-05-01 09:54:57,https://www.chess.com/game/7bf31f6f-079f-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,bsYIpxZRoE2UnD92EM5QxF7ZmCXPgv6XiqZEfxEDltDwem...,7bf31f6f-079f-11ef-9338-b21a8d01000f,,4k3/8/5Q2/P1P1B2P/1P2P3/7B/2K5/6R1 b - -,rapid,...,https://www.chess.com/openings/Van-Geet-Openin...,Van-Geet-Opening-1...c5,Black,591,stalemate,600,stalemate,True,draw,stalemate
1,2024-05-02 02:22:41,https://www.chess.com/game/0b3e5e9f-082a-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mC5Qgv0KfH7TlBWOHQTQBKQCcuZRKR9RbsC0eg6EuB1Tfe...,0b3e5e9f-082a-11ef-a262-7a436f01000f,,5rk1/1pp3pp/p7/1P2N3/P7/2R5/2P2PPP/3r2K1 w - -,rapid,...,https://www.chess.com/openings/Ruy-Lopez-Openi...,Ruy-Lopez-Opening-Gunderam-Variation,White,584,checkmated,621,win,True,lose,checkmated
2,2024-05-04 08:08:17,https://www.chess.com/game/b336281d-09eb-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mCZRgv!TbsYQlBRJCKTCdmCsjs1TcD6EKT2Tmu92ft0KBK...,b336281d-09eb-11ef-8a15-f5237801000f,,5r1k/3Q4/7p/P2Bb2P/8/8/5PP1/2r3K1 w - -,rapid,...,https://www.chess.com/openings/Pirc-Defense-Cz...,Pirc-Defense-Czech-Defense-4.Nf3,White,576,checkmated,597,win,True,lose,checkmated
3,2024-05-04 11:50:54,https://www.chess.com/game/32f6a42f-0a0a-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mCYIbs5QdN2UNIZRIu!Tnv92fH8!HQXQlB6Xgm7Zeg46BJ...,32f6a42f-0a0a-11ef-8a15-f5237801000f,,r7/1P6/1N3Q2/6pk/3B1p1p/P2K4/6PP/2R5 b - -,rapid,...,https://www.chess.com/openings/Closed-Sicilian...,Closed-Sicilian-Defense-Traditional-Line,Black,568,resigned,584,win,True,lose,resigned
4,2024-05-19 12:41:22,https://www.chess.com/game/52dc600d-15db-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mCYIfA0Sbs!Tlt5QnDZRcu2Udm92ec8!pxRJCJSJArJBsJ...,52dc600d-15db-11ef-9d9d-af569601000f,,5rk1/p4p2/6p1/1p3bQp/6PK/3Pr1qP/2PR4/3R4 w - -,rapid,...,https://www.chess.com/openings/Sicilian-Defens...,Sicilian-Defense-Bowdler-Attack-2...e6-3.Nc3,Black,576,win,577,checkmated,True,win,checkmated
5,2024-05-21 03:18:19,https://www.chess.com/game/60480663-1720-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mC0KgvZRfA6Eeg7TAm5ZbsWOsJTVJY80Y4,60480663-1720-11ef-ac29-13086c01000f,,N4bnr/1p1nkppp/p2p3q/4p3/4P1b1/5N2/PPPPBPPP/R1...,rapid,...,https://www.chess.com/openings/Philidor-Defens...,Philidor-Defense-3.Bc4,White,584,win,578,resigned,False,win,resigned
6,2024-05-21 13:56:57,https://www.chess.com/game/9e9051f9-1777-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mC0Kgv5QfA!Tbs9Ieg8!ltZRcuIunu6EtBKBuBEvfvTEBJ...,9e9051f9-1777-11ef-a763-49c05101000f,,6k1/pp6/8/1Kq5/3q4/P7/8/8 w - -,rapid,...,https://www.chess.com/openings/Giuoco-Piano-Ga...,Giuoco-Piano-Game-Four-Knights-Game-5.O-O,White,576,checkmated,590,win,True,lose,checkmated
7,2024-05-21 14:30:39,https://www.chess.com/game/1e22df69-177c-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mC0Kgv5QfHQzegYQvKQHbsZRKv!TiqzQsHTClB7Pqy6Ecu...,1e22df69-177c-11ef-bd50-71056501000f,,1k6/pp2R1p1/1b6/6p1/P1r5/4RK2/1PP5/8 b - -,rapid,...,https://www.chess.com/openings/Ruy-Lopez-Opening,Ruy-Lopez-Opening,White,584,win,588,resigned,True,win,resigned
8,2024-05-22 07:36:34,https://www.chess.com/game/36e9234a-180b-11ef-...,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,True,mCYIgv5QlBIBvBQBdB!TCKTNfm2Ueg92bs8!cuZRfd2KBJ...,36e9234a-180b-11ef-b4d1-05170701000f,,2r1r1k1/3QBpb1/4p1pp/3pq3/1P6/8/P1R1BPPP/3R2K1...,rapid,...,https://www.chess.com/openings/Sicilian-Defens...,Sicilian-Defense-Open-3...cxd4-4.Nxd4,Black,576,timeout,607,win,True,lose,timeout
9,2024-05-22 13:44:02,https://www.chess.com/game/9f2573f4-1840-11ef-...,"[Event ""Live Chess""]\n[Site "